In [1]:
import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, make_scorer, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('../001/train.pk1')

In [3]:
train_pkl.shape

(850, 11)

In [4]:
# 年齢、性別は削除
train_pkl.drop(['Age', 'Gender_Male'], axis=1, inplace=True)

In [5]:
train_pkl.head()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease
0,0.787109,0.150513,220.178696,13.467617,21.729246,6.816406,3.111328,1.006836,0
1,1.003906,0.195679,221.218414,51.033463,64.752319,6.890625,3.050781,0.751465,1
2,0.657227,0.081299,320.770538,12.625011,30.613180,5.949219,2.488281,0.774902,0
3,0.906738,0.214233,369.278168,34.347599,54.510086,6.968750,3.613281,0.988281,1
4,1.735352,0.197754,222.782028,20.572891,170.010178,5.835938,3.068359,1.026367,0


In [6]:
train_pk2 = train_pkl.copy()
train_pkl.shape

(850, 9)

In [7]:
train_pkl = train_pk2.copy()
loop = train_pkl.shape[1] - 1
for i in range(loop):
    for j in range(i+1, loop):
        colname = train_pkl.columns[i] + "_" + train_pkl.columns[j] 
        train_pkl["mul" + "_" + colname] = train_pkl.iloc[:, i] * train_pkl.iloc[:, j]

In [8]:
train_pkl.head()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease,mul_T_Bil_D_Bil,...,mul_ALT_GPT_AST_GOT,mul_ALT_GPT_TP,mul_ALT_GPT_Alb,mul_ALT_GPT_AG_ratio,mul_AST_GOT_TP,mul_AST_GOT_Alb,mul_AST_GOT_AG_ratio,mul_TP_Alb,mul_TP_AG_ratio,mul_Alb_AG_ratio
0,0.787109,0.150513,220.178696,13.467617,21.729246,6.816406,3.111328,1.006836,0,0.118469,...,292.641174,91.800751,41.902176,13.559681,148.115372,67.606812,21.877787,21.203125,6.863281,3.132812
1,1.003906,0.195679,221.218414,51.033463,64.752319,6.890625,3.050781,0.751465,1,0.196411,...,3304.535156,351.652466,155.691925,38.349854,446.183960,197.545166,48.659092,21.015625,5.179688,2.292969
2,0.657227,0.081299,320.770538,12.625011,30.613180,5.949219,2.488281,0.774902,0,0.053436,...,386.491760,75.108955,31.414579,9.783151,182.124512,76.174202,23.722225,14.804688,4.609375,1.927734
3,0.906738,0.214233,369.278168,34.347599,54.510086,6.968750,3.613281,0.988281,1,0.194214,...,1872.290527,239.359833,124.107536,33.945087,379.867157,196.960266,53.871296,25.187500,6.886719,3.570312
4,1.735352,0.197754,222.782028,20.572891,170.010178,5.835938,3.068359,1.026367,0,0.343262,...,3497.600830,120.062111,63.125023,21.115341,992.168762,521.652344,174.492874,17.906250,5.988281,3.148438


In [9]:
train_pkl.describe()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease,mul_T_Bil_D_Bil,...,mul_ALT_GPT_AST_GOT,mul_ALT_GPT_TP,mul_ALT_GPT_Alb,mul_ALT_GPT_AG_ratio,mul_AST_GOT_TP,mul_AST_GOT_Alb,mul_AST_GOT_AG_ratio,mul_TP_Alb,mul_TP_AG_ratio,mul_Alb_AG_ratio
count,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,...,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000
mean,1.607422,0.606445,272.211334,41.978474,61.018120,7.031250,3.517578,1.144531,0.444706,3.859375,...,13083.430664,282.884216,140.503342,48.240143,422.477356,209.288589,69.282700,25.078125,8.101562,4.105469
std,2.443359,1.568359,200.348511,145.927368,110.125389,0.840820,0.560059,0.227295,0.497226,inf,...,86126.578125,907.551208,451.523315,170.731842,756.458435,375.404877,134.365646,6.441406,2.060547,1.321289
min,0.585938,0.034851,163.261841,3.924518,11.283497,4.859375,2.181641,0.626953,0.000000,0.023529,...,80.279144,27.563610,13.682158,5.081944,71.421677,34.344650,11.312227,11.625000,3.632812,1.509766
25%,0.781372,0.144928,213.991409,13.644659,21.239157,6.730469,3.138672,1.004883,0.000000,0.117188,...,305.416222,95.244083,47.700649,14.526553,152.569931,74.945957,25.734465,21.082031,6.694336,3.183594
50%,0.835693,0.194092,220.092506,16.638000,27.056026,6.914062,3.621094,1.205078,0.000000,0.161621,...,513.703537,117.457916,57.821743,19.545650,186.471611,99.210655,31.773411,25.046875,7.990234,3.959961
75%,1.197021,0.335388,229.455933,23.056081,56.461569,7.535156,3.712891,1.288086,1.000000,0.414490,...,1182.776764,162.720444,84.048071,27.486331,393.104301,189.982578,58.352411,27.871094,9.654297,4.699219
max,23.015625,17.687500,2108.483643,1423.186523,814.439392,8.742188,5.015625,1.821289,1.000000,407.000000,...,912347.875000,8605.831055,4491.932617,1713.363525,6165.100586,3218.158936,1243.772949,43.781250,15.773438,9.085938


In [10]:
disease_bk = train_pkl.disease.values

In [11]:
train_pkl.drop('disease', axis=1, inplace=True)

In [12]:
train_pkl['disease'] = disease_bk

In [13]:
train_pkl.head()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,mul_T_Bil_D_Bil,mul_T_Bil_ALP,...,mul_ALT_GPT_TP,mul_ALT_GPT_Alb,mul_ALT_GPT_AG_ratio,mul_AST_GOT_TP,mul_AST_GOT_Alb,mul_AST_GOT_AG_ratio,mul_TP_Alb,mul_TP_AG_ratio,mul_Alb_AG_ratio,disease
0,0.787109,0.150513,220.178696,13.467617,21.729246,6.816406,3.111328,1.006836,0.118469,173.304718,...,91.800751,41.902176,13.559681,148.115372,67.606812,21.877787,21.203125,6.863281,3.132812,0
1,1.003906,0.195679,221.218414,51.033463,64.752319,6.890625,3.050781,0.751465,0.196411,222.082550,...,351.652466,155.691925,38.349854,446.183960,197.545166,48.659092,21.015625,5.179688,2.292969,1
2,0.657227,0.081299,320.770538,12.625011,30.613180,5.949219,2.488281,0.774902,0.053436,210.818924,...,75.108955,31.414579,9.783151,182.124512,76.174202,23.722225,14.804688,4.609375,1.927734,0
3,0.906738,0.214233,369.278168,34.347599,54.510086,6.968750,3.613281,0.988281,0.194214,334.838654,...,239.359833,124.107536,33.945087,379.867157,196.960266,53.871296,25.187500,6.886719,3.570312,1
4,1.735352,0.197754,222.782028,20.572891,170.010178,5.835938,3.068359,1.026367,0.343262,386.605133,...,120.062111,63.125023,21.115341,992.168762,521.652344,174.492874,17.906250,5.988281,3.148438,0


## 訓練データとテストデータに分割

In [14]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [15]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, :-1]
train_y = train.disease.values

In [16]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, :-1]
test_y = test.disease.values

## 訓練データで訓練

In [147]:
XGB = XGBClassifier(lambda_l1=300, lamba_l2=300, max_depth=8)

In [148]:
XGB = XGB.fit(train_X, train_y, early_stopping_rounds=20, eval_metric='auc', eval_set=[(test_x, test_y)])

[0]	validation_0-auc:0.918587
Will train until validation_0-auc hasn't improved in 20 rounds.
[1]	validation_0-auc:0.932132
[2]	validation_0-auc:0.938766
[3]	validation_0-auc:0.940371
[4]	validation_0-auc:0.94484
[5]	validation_0-auc:0.943932
[6]	validation_0-auc:0.941838
[7]	validation_0-auc:0.941558
[8]	validation_0-auc:0.945399
[9]	validation_0-auc:0.949029
[10]	validation_0-auc:0.948471
[11]	validation_0-auc:0.946097
[12]	validation_0-auc:0.948471
[13]	validation_0-auc:0.946516
[14]	validation_0-auc:0.949448
[15]	validation_0-auc:0.949728
[16]	validation_0-auc:0.949588
[17]	validation_0-auc:0.951264
[18]	validation_0-auc:0.952521
[19]	validation_0-auc:0.953219
[20]	validation_0-auc:0.953638
[21]	validation_0-auc:0.954336
[22]	validation_0-auc:0.956012
[23]	validation_0-auc:0.956291
[24]	validation_0-auc:0.957129
[25]	validation_0-auc:0.957967
[26]	validation_0-auc:0.959084
[27]	validation_0-auc:0.958106
[28]	validation_0-auc:0.958106
[29]	validation_0-auc:0.959084
[30]	validation_0

In [149]:
XGB.feature_importances_

array([0.01500495, 0.01373418, 0.01404946, 0.01328435, 0.01186019,
       0.00885388, 0.00813905, 0.01819055, 0.01303941, 0.02055771,
       0.07055584, 0.39271435, 0.02226604, 0.01627987, 0.02022654,
       0.0085704 , 0.0370657 , 0.01992338, 0.0166044 , 0.00762197,
       0.0160112 , 0.01827734, 0.01517834, 0.0105479 , 0.01060939,
       0.01073577, 0.00586083, 0.00920723, 0.01302872, 0.01193038,
       0.02063597, 0.02414905, 0.02764022, 0.00639143, 0.01569541,
       0.03555873], dtype=float32)

In [150]:
sorted(
    zip(map(lambda x: round(x, 3), XGB.feature_importances_), train_X.iloc[:, :].columns),
    reverse=True)

[(0.393, 'mul_T_Bil_AST_GOT'),
 (0.071, 'mul_T_Bil_ALT_GPT'),
 (0.037, 'mul_D_Bil_ALT_GPT'),
 (0.036, 'mul_Alb_AG_ratio'),
 (0.028, 'mul_AST_GOT_AG_ratio'),
 (0.024, 'mul_AST_GOT_Alb'),
 (0.022, 'mul_T_Bil_TP'),
 (0.021, 'mul_T_Bil_ALP'),
 (0.021, 'mul_AST_GOT_TP'),
 (0.02, 'mul_T_Bil_AG_ratio'),
 (0.02, 'mul_D_Bil_AST_GOT'),
 (0.018, 'mul_ALP_ALT_GPT'),
 (0.018, 'AG_ratio'),
 (0.017, 'mul_D_Bil_TP'),
 (0.016, 'mul_T_Bil_Alb'),
 (0.016, 'mul_TP_AG_ratio'),
 (0.016, 'mul_D_Bil_AG_ratio'),
 (0.015, 'mul_ALP_AST_GOT'),
 (0.015, 'T_Bil'),
 (0.014, 'D_Bil'),
 (0.014, 'ALP'),
 (0.013, 'mul_T_Bil_D_Bil'),
 (0.013, 'mul_ALT_GPT_Alb'),
 (0.013, 'ALT_GPT'),
 (0.012, 'mul_ALT_GPT_AG_ratio'),
 (0.012, 'AST_GOT'),
 (0.011, 'mul_ALP_TP'),
 (0.011, 'mul_ALP_Alb'),
 (0.011, 'mul_ALP_AG_ratio'),
 (0.009, 'mul_D_Bil_ALP'),
 (0.009, 'mul_ALT_GPT_TP'),
 (0.009, 'TP'),
 (0.008, 'mul_D_Bil_Alb'),
 (0.008, 'Alb'),
 (0.006, 'mul_TP_Alb'),
 (0.006, 'mul_ALT_GPT_AST_GOT')]

## テストデータで実行

In [151]:
test_x.shape, test_y.shape

((170, 36), (170,))

In [152]:
pred_y = XGB.predict(test_x)

In [153]:
confusion_matrix(test_y, pred_y)

array([[88,  5],
       [15, 62]], dtype=int64)

In [154]:
accuracy_score(test_y, pred_y)

0.8823529411764706

## 検証データで実行

In [115]:
# 検証データ読み込み
valid = pd.read_pickle('../001/test.pk1')

In [116]:
valid.shape

(350, 10)

In [117]:
type(valid)

pandas.core.frame.DataFrame

In [124]:
# valid_X = valid.iloc[:, 1:]
valid_X = valid.copy()

In [125]:
valid_X.head()

,Age,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,Gender_Male
0,25,0.801758,0.186768,214.448685,15.260516,19.496649,6.816406,3.601562,1.288086,1
1,26,0.834961,0.116455,234.136810,10.509159,16.733311,6.796875,3.111328,1.030273,0
2,64,0.791992,0.197998,216.039902,14.578304,20.695866,7.523438,3.626953,1.278320,1
3,65,0.833984,0.175781,200.901123,12.820356,20.102516,6.828125,3.626953,1.275391,1
4,34,0.690430,0.135498,208.705841,20.279018,25.096685,8.664062,4.171875,1.280273,0


In [126]:
# 年齢、性別は削除
valid_X.drop(['Age', 'Gender_Male'], axis=1, inplace=True)

In [127]:
valid_X.shape

(350, 8)

In [128]:
loop = valid_X.shape[1]
for i in range(loop):
    for j in range(i+1, loop):
        colname = valid_X.columns[i] + "_" + valid_X.columns[j] 
        valid_X["mul" + "_" + colname] = valid_X.iloc[:, i] * valid_X.iloc[:, j]

In [129]:
valid_X.shape, train_X.shape

((350, 36), (680, 36))

In [132]:
valid_X.head()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,mul_T_Bil_D_Bil,mul_T_Bil_ALP,...,mul_ALT_GPT_AST_GOT,mul_ALT_GPT_TP,mul_ALT_GPT_Alb,mul_ALT_GPT_AG_ratio,mul_AST_GOT_TP,mul_AST_GOT_Alb,mul_AST_GOT_AG_ratio,mul_TP_Alb,mul_TP_AG_ratio,mul_Alb_AG_ratio
0,0.801758,0.186768,214.448685,15.260516,19.496649,6.816406,3.601562,1.288086,0.149780,171.935913,...,297.528931,104.021881,54.961704,19.656857,132.897079,70.218399,25.113359,24.546875,8.781250,4.640625
1,0.834961,0.116455,234.136810,10.509159,16.733311,6.796875,3.111328,1.030273,0.097229,195.495087,...,175.853027,71.429443,32.697441,10.827308,113.734222,52.062820,17.239885,21.140625,7.003906,3.205078
2,0.791992,0.197998,216.039902,14.578304,20.695866,7.523438,3.626953,1.278320,0.156860,171.101913,...,301.710632,109.678963,52.874825,18.635742,155.704056,75.062935,26.455946,27.281250,9.617188,4.636719
3,0.833984,0.175781,200.901123,12.820356,20.102516,6.828125,3.626953,1.275391,0.146606,167.548401,...,257.721436,87.538994,46.498833,16.350962,137.262497,72.910881,25.638561,24.765625,8.710938,4.625000
4,0.690430,0.135498,208.705841,20.279018,25.096685,8.664062,4.171875,1.280273,0.093567,144.096710,...,508.936157,175.698685,84.601532,25.962688,217.439255,104.700233,32.130619,36.156250,11.093750,5.339844


In [133]:
pred_valid_y = XGB.predict(valid_X)

In [134]:
pred_valid_y.shape

(350,)

In [135]:
# type(valid_pass), type(pred_valid_y)

In [136]:
result_df = pd.DataFrame(pred_valid_y)

In [137]:
result_df.to_csv("./XGB_6.csv", header=False)

In [138]:
pred_valid_y.sum()

139